<a href="https://colab.research.google.com/github/MohanKumar2002/Translation_Model-using-Mariaan-model/blob/master/Translation_model_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fine Tune and Save Model

Install Libraries:

In [ ]:
!pip install transformers datasets sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
   

Load and Prepare Data:

In [ ]:
import pandas as pd

# Adjust the path if necessary
data_path = '/content/Translator_data.csv'
df = pd.read_csv(data_path)
print(df.head())


  English Sentence           Hindi          Bengali             Telugu  \
0    Hello, world!  ?????? ??????!   ??????, ?????!      ???, ???????!   
1     How are you?    ?? ???? ????  ???? ???? ?????  ???? ??? ????????   
2     Good morning      ??? ??????         ????????            ???????   
3        Thank you         ???????          ???????         ??????????   
4              Yes             ???            ?????              ?????   

         Marathi                           Tamil        Gujarati  \
0   ???????, ??!                 ???????, ?????!  ??????, ?????!   
1  ??? ??? ?????  ??????? ?????? ???????????????     ??? ??? ???   
2     ??? ??????                    ???? ???????        ????????   
3        ???????                           ?????            ????   
4            ???                             ???              ??   

           Kannada                    Malayalam                Odia  \
0     ???, ??????!                   ???, ????!        ????, ?????!   
1  ?

In [ ]:
import json

data = []
for index, row in df.iterrows():
    for lang in df.columns[1:]:
        data.append({"src": row["English Sentence"], "tgt": row[lang], "src_lang": "en", "tgt_lang": lang})

with open('/content/translator_data.jsonl', 'w') as outfile:
    for entry in data:
        json.dump(entry, outfile)
        outfile.write('\n')


Load and Tokenize Data:

In [ ]:
from datasets import load_dataset, Dataset
#!pip install transformers
from transformers import MarianMTModel, MarianTokenizer

dataset = load_dataset('json', data_files='/content/translator_data.jsonl', split='train')

def preprocess_function(examples):
    src_texts = examples['src']
    tgt_texts = examples['tgt']
    model_inputs = tokenizer(src_texts, max_length=128, truncation=True, padding="max_length") # Added padding="max_length" to ensure all sequences are the same length
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(tgt_texts, max_length=128, truncation=True, padding="max_length") # Added padding="max_length" to ensure all sequences are the same length
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

tokenizer = MarianTokenizer.from_pretrained('Helsinki-NLP/opus-mt-en-ROMANCE') #The MarianTokenizer class is part of the transformers library
tokenized_dataset = dataset.map(preprocess_function, batched=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Map:   0%|          | 0/400 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4016: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Fine-Tune the Model:

In [ ]:
from transformers import MarianMTModel, Trainer, TrainingArguments

model = MarianMTModel.from_pretrained('Helsinki-NLP/opus-mt-en-ROMANCE')

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    logging_dir='./logs',
    logging_steps=200,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

trainer.train()


Step,Training Loss
200,0.105400
400,0.054300


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[65000]], 'forced_eos_token_id': 0}


TrainOutput(global_step=500, training_loss=0.07402849006652833, metrics={'train_runtime': 3475.1453, 'train_samples_per_second': 0.576, 'train_steps_per_second': 0.144, 'total_flos': 67796729856000.0, 'train_loss': 0.07402849006652833, 'epoch': 5.0})

Save the Fine-Tuned Model:

In [ ]:
model.save_pretrained('/content/translator_model')
tokenizer.save_pretrained('/content/translator_model')


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[65000]], 'forced_eos_token_id': 0}


('/content/translator_model/tokenizer_config.json',
 '/content/translator_model/special_tokens_map.json',
 '/content/translator_model/vocab.json',
 '/content/translator_model/source.spm',
 '/content/translator_model/target.spm',
 '/content/translator_model/added_tokens.json')

Steps to Create a Copycat Interface of Google Translator

Install Required Libraries:

In [ ]:
!pip install streamlit pyngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.1 MB/s eta 0:00:00


Create the Streamlit App Script:

In [ ]:
%%writefile app.py
import streamlit as st
from transformers import MarianMTModel, MarianTokenizer

# Load the model and tokenizer
model_name = 'Helsinki-NLP/opus-mt-en-ROMANCE'
model = MarianMTModel.from_pretrained('/content/drive/MyDrive/translator_model')
tokenizer = MarianTokenizer.from_pretrained('/content/drive/MyDrive/translator_model')

# Streamlit interface
st.title("Copycat Google Translator")

src_text = st.text_area("Enter text in English", "")
if st.button("Translate"):
    if src_text:
        inputs = tokenizer(src_text, return_tensors="pt", padding=True)
        translated = model.generate(**inputs)
        tgt_text = tokenizer.decode(translated[0], skip_special_tokens=True)
        st.text_area("Translated text", tgt_text)
    else:
        st.warning("Please enter text to translate.")


Writing app.py


Run the Streamlit App:

In [ ]:
from pyngrok import ngrok

# Run the Streamlit app
!streamlit run app.py &

# Create a public URL
public_url = ngrok.connect(port='8501')
print(f"Streamlit app is live at: {public_url}")


Testing

In [ ]:
from transformers import MarianMTModel, MarianTokenizer

# Load the model and tokenizer
model_path = '/content/translator_model'
model = MarianMTModel.from_pretrained(model_path)
tokenizer = MarianTokenizer.from_pretrained(model_path)

def translate(text, target_lang):
    # Prepare the inputs
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

    # Generate translation
    translated = model.generate(**inputs)

    # Decode the translated text
    tgt_text = tokenizer.decode(translated[0], skip_special_tokens=True)
    return tgt_text

# Example sentences to translate
sentences = [
    "Can you help me?",
    "I am lost",
    "Where is the nearest hospital?",
    "Where is the train station?",
    "Is it far?",
    "I don't know",
    "Can I have some water?",
    "Can you show me the way?",
    "Where is the nearest ATM?",
    "Do you speak English?",
    "Where is the bus stop?",
    "I am sorry to hear that"
]

# Languages corresponding to columns in the CSV
languages = [
    'hi', 'bn', 'te', 'mr', 'ta', 'gu', 'ml', 'kn', 'ml', 'or', 'pa'
]

# Test translations
for lang in languages:
    print(f"\nTranslating to language code: {lang}")
    for sentence in sentences:
        translated_sentence = translate(sentence, lang)
        print(f"{sentence} -> {translated_sentence}")


/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")



Translating to language code: hi
Can you help me? -> ????????????????????????
I am lost -> ????????????????????????????
Where is the nearest hospital? -> ????????????????????????
Where is the train station? -> ?????????????????????
Is it far? -> ??????????
I don't know -> ??????????????
Can I have some water? -> ???????????????????????????????????????????????????
Can you show me the way? -> ???????????????????????????????????????????????????????
Where is the nearest ATM? -> ??????????????????????
Do you speak English? -> ????????????????????????????????????????????????????????????????
Where is the bus stop? -> ??????????????????
I am sorry to hear that -> ????????????????????????????

Translating to language code: bn
Can you help me? -> ????????????????????????
I am lost -> ????????????????????????????
Where is the nearest hospital? -> ????????????????????????
Where is the train station? -> ?????????????????????
Is it far? -> ??????????
I don't know -> ??????????????
Can I have some w

Another way

In [ ]:
from transformers import MarianMTModel, MarianTokenizer

# Load the model and tokenizer
model_path = '/content/translator_model'
model = MarianMTModel.from_pretrained(model_path)
tokenizer = MarianTokenizer.from_pretrained(model_path)


In [ ]:
def translate(text, target_lang):
    # Prepare the inputs
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

    # Generate translation
    translated = model.generate(**inputs)

    # Decode the translated text
    tgt_text = tokenizer.decode(translated[0], skip_special_tokens=True)
    return tgt_text


In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Define language options (use language codes that your model supports)
language_options = {
    'Hindi': 'hi',
    'Bengali': 'bn',
    'Telugu': 'te',
    'Marathi': 'mr',
    'Tamil': 'ta',
    'Gujarati': 'gu',
    'Malayalam': 'ml',
    'Kannada': 'kn',
    'Oriya': 'or',
    'Punjabi': 'pa'
}

# Create widgets
text_widget = widgets.Text(
    description='Input Text:',
    placeholder='Enter text here',
    style={'description_width': 'initial'}
)

language_widget = widgets.Dropdown(
    options=language_options,
    description='Target Language:',
    style={'description_width': 'initial'}
)

output_widget = widgets.Output()

def on_button_click(b):
    with output_widget:
        # Clear previous output
        output_widget.clear_output()

        # Get the text and selected language
        text = text_widget.value
        target_lang = language_widget.value

        if text and target_lang:
            # Perform translation
            translated_text = translate(text, target_lang)
            print(f"Original Text: {text}")
            print(f"Translated Text: {translated_text}")
        else:
            print("Please provide both input text and select a target language.")

# Create a button
button = widgets.Button(description="Translate")
button.on_click(on_button_click)

# Display widgets
display(text_widget, language_widget, button, output_widget)


Text(value='', description='Input Text:', placeholder='Enter text here', style=DescriptionStyle(description_wi…

Dropdown(description='Target Language:', options={'Hindi': 'hi', 'Bengali': 'bn', 'Telugu': 'te', 'Marathi': '…

Button(description='Translate', style=ButtonStyle())

Output()

Complete Code to Zip and Download the Model

In [ ]:
from transformers import MarianMTModel, MarianTokenizer
import shutil
from google.colab import files

# Define the path where the model and tokenizer will be saved
model_path = '/content/translator_model'

# Zip the model directory
shutil.make_archive('/content/translator_model', 'zip', model_path)

# Download the zip file
files.download('/content/translator_model.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>